In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import pandas as pd
import numpy as np

from PIL import Image
import os

In [2]:
# Load datasets
train_df = pd.read_csv('./dataset/train.csv')
test_df = pd.read_csv('./dataset/test.csv')

path_prefix = "./dataset/new_imageset/"

# Original categories
original_categories = [3, 5, 7, 11, 16, 17, 26, 28, 35, 43, 54, 55]

# Create a mapping from original categories to zero-indexed labels
label_mapping = {cat: idx for idx, cat in enumerate(original_categories)}


In [3]:
class TrafficSignDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None, label_mapping=None):
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform
        self.label_mapping = label_mapping

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.dataframe.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        original_label = int(self.dataframe.iloc[idx, 1])
        label = self.label_mapping[original_label]  # Use the mapped label

        if self.transform:
            image = self.transform(image)
        
        return image, label

In [4]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets using the label mapping
train_dataset = TrafficSignDataset(train_df, path_prefix, transform=transform, label_mapping=label_mapping)
test_dataset = TrafficSignDataset(test_df, path_prefix, transform=transform, label_mapping=label_mapping)

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [5]:
class TrafficSignCNN(nn.Module):
    def __init__(self):
        super(TrafficSignCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2)  
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 32 * 32, 1000)
        self.fc2 = nn.Linear(1000, len(label_mapping))
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.maxpool(x)
        x = self.relu(self.conv2(x))
        x = self.maxpool(x)
        x = x.view(-1, 64 * 32 * 32)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TrafficSignCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 25
for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

torch.save(model.state_dict(), 'traffic_sign_model_state_dict.pth')

Epoch 1, Loss: 0.12536561489105225
Epoch 2, Loss: 0.012740697711706161
Epoch 3, Loss: 0.014430709183216095
Epoch 4, Loss: 6.004097485856619e-06
Epoch 5, Loss: 0.005064811557531357
Epoch 6, Loss: 0.00012272542517166585
Epoch 7, Loss: 7.773381184961181e-06
Epoch 8, Loss: 2.145743792425492e-06
Epoch 9, Loss: 0.0008070935145951807
Epoch 10, Loss: 0.00039699123590253294
Epoch 11, Loss: 5.878613956156187e-05
Epoch 12, Loss: 2.1349103917600587e-05
Epoch 13, Loss: 1.792273178580217e-05
Epoch 14, Loss: 8.193747817131225e-06
Epoch 15, Loss: 6.860466237412766e-05
Epoch 16, Loss: 1.6260974007309414e-05
Epoch 17, Loss: 2.729230573095265e-06
Epoch 18, Loss: 0.00012153424177085981
Epoch 19, Loss: 1.7397189367329702e-05
Epoch 20, Loss: 9.128302190219983e-05
Epoch 21, Loss: 1.0803786608448718e-05
Epoch 22, Loss: 7.089407154126093e-05
Epoch 23, Loss: 2.2603737306781113e-05
Epoch 24, Loss: 7.985239062691107e-05
Epoch 25, Loss: 4.436938252183609e-05


In [10]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy: {100 * correct / total}%')

Accuracy: 99.70457902511079%


In [8]:
print(model)

TrafficSignCNN(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=65536, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=12, bias=True)
  (relu): ReLU()
)
